In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "32"

In [2]:
from graph_tool.all import *
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.covariance import LedoitWolf, OAS
#import matplotlib.pyplot as py
#import seaborn as sns
import statsmodels.api as sm
from multipy.fdr import qvalue
from multipy.fdr import lsu

In [6]:
import trim_networks
import fit_sbm

In [3]:
### Input expression data
gene_expr_body = pd.read_table("/Genomics/ayroleslab2/lamaya/bigProject/eQTLcatalog/modularity/matrices/VOOMCounts_CPM5_counts4M_covfree_body_ctrl_onlygenesinmainchr_Jul20.21.txt")
gene_expr_head = pd.read_table("/Genomics/ayroleslab2/lamaya/bigProject/eQTLcatalog/modularity/matrices/VOOMCounts_CPM5_counts4M_covfree_head_ctrl_onlygenesinmainchr_Jul20.21.txt")

In [4]:
gene_expr_dict = {"head": gene_expr_head.T, "body": gene_expr_body.T}

In [6]:
g_body = load_graph("../data/output/SBM/graphs/VOOMCounts_CPM1_body_ctrl_onlygenesinmainchr_939ind_covfree_Jul21.21.xml.gz")
g_head = load_graph("../data/output/SBM/graphs/VOOMCounts_CPM1_head_ctrl_onlygenesinmainchr_940ind_covfree_Jul21.21.xml.gz")

g_dict = {"head": g_head, "body": g_body}

In [11]:
def getGeneNetworkStats(g, tissue):
    genes = g.vertex_properties["genes"]
    corr = g.edge_properties["spearman"]
    block_df = pd.DataFrame(columns=('Gene', 'Tissue', "Average_Spearman",
                                     'Degree_thr_0.1', 'WeightedDegree_thr_0.1',
                                     'Degree_thr_0.2', 'WeightedDegree_thr_0.2',
                                     'Degree_thr_0.3', 'WeightedDegree_thr_0.3',
                                     'Degree_thr_0.4', 'WeightedDegree_thr_0.4',
                                     'Degree_thr_0.5', 'WeightedDegree_thr_0.5',
                                     'Degree_fdr_0.05'     , 'WeightedDegree_fdr_0.05',
                                     'Degree_fdr_0.01'     , 'WeightedDegree_fdr_0.01',
                                     'Degree_fdr_0.001'     , 'WeightedDegree_fdr_0.001',
                                     'Degree_fdr_0.0001'     , 'WeightedDegree_fdr_0.0001'))

    tv_01 = filterByEdge(g, "spearman", 0.1, False)
    tv_02 = filterByEdge(g, "spearman", 0.2, False)
    tv_03 = filterByEdge(g, "spearman", 0.3, False)
    tv_04 = filterByEdge(g, "spearman", 0.4, False)
    tv_05 = filterByEdge(g, "spearman", 0.5, False)

    tv_fdr05 = filterByFDR(g, 0.05, True)
    tv_fdr01 = filterByFDR(g, 0.01, True)
    tv_fdr001 = filterByFDR(g, 0.001, True)
    tv_fdr0001 = filterByFDR(g, 0.0001, True)

    for v in g.vertex_index:
        line = [genes[v]]
        line.append(tissue)
        line.append(np.mean(np.abs(g.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_01.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_01.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_02.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_02.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_03.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_03.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_04.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_04.get_all_edges(v, [corr] )[:,2])))
        
        line.append(tv_05.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_05.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_fdr05.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr05.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_fdr01.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr01.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_fdr001.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr001.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_fdr0001.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr0001.get_all_edges(v, [corr] )[:,2])))

        block_df.loc[v] = line
    return block_df


In [21]:
def saveGeneTable(g, gene_expr, output=None):
    gene_list = []
    for i in g.vertex_properties['genes']:
        gene_list.append(i)
    expr_df = gene_expr[gene_list]
    if output is not None:
        expr_df.to_csv(output)
    return expr_df

In [12]:
gene_stats = {"body": getGeneNetworkStats(g_dict["body"], "body"), 
              "head": getGeneNetworkStats(g_dict["head"], "head") }

/Genomics/grid/users/damelo/.conda/envs/gt/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Genomics/grid/users/damelo/.conda/envs/gt/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
gene_stats["body"].append(gene_stats["head"])

,Gene,Tissue,Average_Spearman,Degree_thr_0.1,WeightedDegree_thr_0.1,Degree_thr_0.2,WeightedDegree_thr_0.2,Degree_thr_0.3,WeightedDegree_thr_0.3,Degree_thr_0.4,...,Degree_thr_0.5,WeightedDegree_thr_0.5,Degree_fdr_0.05,WeightedDegree_fdr_0.05,Degree_fdr_0.01,WeightedDegree_fdr_0.01,Degree_fdr_0.001,WeightedDegree_fdr_0.001,Degree_fdr_0.0001,WeightedDegree_fdr_0.0001
0,FBgn0031081,body,0.038856,283,0.114046,0,NaN,0,NaN,0,...,0,NaN,634,0.097215,212,0.113119,39,0.129258,9,0.144224
1,FBgn0031080,body,0.033612,131,0.114012,0,NaN,0,NaN,0,...,0,NaN,223,0.098251,38,0.115955,2,0.142527,1,0.163184
2,FBgn0053217,body,0.028115,32,0.109037,0,NaN,0,NaN,0,...,0,NaN,146,0.086197,24,0.101857,1,0.126439,0,NaN
3,FBgn0052350,body,0.056061,1281,0.132787,38,0.216123,0,NaN,0,...,0,NaN,1995,0.112259,1218,0.127616,687,0.145256,420,0.159794
4,FBgn0024733,body,0.112493,3621,0.196767,1194,0.307042,371,0.459460,164,...,106,0.709734,4104,0.182957,3299,0.203641,2531,0.229351,2036,0.250166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8646,FBgn0031309,head,0.031905,93,0.113254,0,NaN,0,NaN,0,...,0,NaN,292,0.087731,81,0.103321,12,0.118972,2,0.108961
8647,FBgn0031305,head,0.094757,2816,0.203328,1023,0.313695,447,0.407724,223,...,63,0.531027,2726,0.197394,2049,0.228154,1524,0.259945,1230,0.284257
8648,FBgn0016926,head,0.054022,1227,0.136051,71,0.222784,0,NaN,0,...,0,NaN,1569,0.121740,915,0.140344,494,0.159659,270,0.179268
8649,FBgn0031299,head,0.039226,340,0.117233,0,NaN,0,NaN,0,...,0,NaN,692,0.100287,263,0.114417,63,0.135549,25,0.149055


In [14]:
gene_stats["body"].append(gene_stats["head"]).to_csv("../data/output/connectivity_stats_VOOMCounts_CPM1_headbody_ctrl_onlygenesinmainchr_covfree_Jul21.21.csv")

In [23]:
g_body = load_graph("../data/output/SBM/graphs/body_fdrLevel-1e-05_genes-3407_density-0.042.xml.gz")
g_head = load_graph("../data/output/SBM/graphs/head_fdrLevel-1e-05_genes-4893_density-0.024.xml.gz")

In [26]:
saveGeneTable(g_body, gene_expr_dict["body"], "body_table_wcgna_fdrLevel-1e-05.csv")

,FBgn0024733,FBgn0040372,FBgn0023537,FBgn0025640,FBgn0003575,FBgn0025638,FBgn0040383,FBgn0025621,FBgn0015799,FBgn0015288,...,FBgn0000579,FBgn0021906,FBgn0053126,FBgn0040723,FBgn0031320,FBgn0031313,FBgn0031309,FBgn0031305,FBgn0016926,FBgn0003310
108_C8,12.695032,6.000547,7.338484,5.980783,4.898900,8.040572,4.605420,4.500883,5.951868,6.698181,...,10.39728,8.854291,6.579661,6.103005,3.532407,5.911469,4.745208,4.759798,7.339688,5.703859
112_A12,12.559422,6.654485,7.541150,6.949690,4.685937,8.223292,4.926830,3.337411,5.648729,7.742281,...,10.76636,9.107028,5.862391,4.809238,1.909671,6.174153,4.728252,4.619091,7.190200,6.547516
112_B12,12.726410,7.031838,7.352289,5.498161,5.177109,8.218612,5.228890,5.659689,6.208262,5.889386,...,10.29173,8.724388,6.365549,6.154048,6.057163,6.366375,5.803887,5.492252,7.437489,6.510155
112_D7,12.797517,6.111535,7.141141,5.916476,4.768836,7.497395,5.547144,3.805370,7.167270,6.557405,...,10.25550,9.107853,6.466491,6.175410,3.102601,6.095470,4.742845,4.432477,7.478031,4.838137
112_E12,12.257089,5.452908,7.381344,5.447582,4.616593,7.202188,5.054254,4.959335,6.330210,6.758847,...,10.12573,8.480394,6.174665,5.893540,2.915394,6.143625,4.876625,5.258968,7.878135,6.553753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25_H8,12.754213,5.834524,7.004133,6.833859,4.929010,8.376495,4.923010,4.641277,5.342262,6.917752,...,10.38828,9.030026,6.796075,4.788802,2.996571,6.003629,4.152469,3.976340,7.290694,5.183859
25_H7,12.485194,6.065159,6.996279,5.814590,4.689909,8.126452,4.379438,4.779917,6.309005,6.489891,...,10.30450,8.801357,6.196780,6.051301,3.466903,5.781781,4.934488,5.143842,7.044730,5.584516
25_H6,12.949873,5.697182,7.059080,5.667081,5.127694,8.043379,4.710522,3.951176,5.598540,6.492429,...,10.35760,9.116645,7.158598,7.294205,2.929750,5.749880,4.860974,4.650204,7.565679,4.980945
25_H5,12.695900,5.910896,7.524204,5.297758,4.035707,8.126555,4.398904,3.726243,4.718881,6.446165,...,10.61232,9.267198,6.761414,6.835787,4.112525,6.650797,5.923766,5.652081,6.966681,5.970549


In [27]:
saveGeneTable(g_head, gene_expr_dict["head"], "head_table_wcgna_fdrLevel-1e-05.csv")

,FBgn0031081,FBgn0052350,FBgn0024733,FBgn0040372,FBgn0023534,FBgn0023537,FBgn0000108,FBgn0025640,FBgn0003575,FBgn0025634,...,FBgn0028481,FBgn0031304,FBgn0031306,FBgn0031310,FBgn0031312,FBgn0031318,FBgn0031313,FBgn0031305,FBgn0016926,FBgn0003310
106_B7,5.060509,4.705684,12.416530,3.760163,3.591924,7.611656,8.750688,5.015808,4.307134,3.562829,...,5.241275,3.555108,7.110114,5.481513,4.799607,5.252070,5.435192,5.508806,7.425574,4.828373
106_C6,5.297734,3.221103,12.212429,4.384999,3.944236,7.215610,9.191008,5.290460,4.140848,3.746981,...,5.244818,3.851083,7.523898,4.449949,5.350948,4.813783,7.312520,4.920895,8.226443,4.679178
106_C8,5.499218,3.077922,12.000662,3.878972,4.542412,7.675215,9.572163,4.927570,3.967215,3.738268,...,6.083417,3.456773,7.661656,3.873471,4.812516,5.284459,7.094768,5.114885,7.703878,4.257941
106_A2,5.144081,3.187525,11.504131,4.137904,4.274401,7.459486,9.281683,5.175146,4.021908,3.874211,...,5.329936,3.742363,7.530514,4.999761,4.846335,4.977347,5.775082,4.896231,8.044966,4.244092
106_G4,4.608946,4.985678,11.611643,3.705962,4.066724,6.956967,9.231659,5.675135,5.173601,3.027501,...,5.423124,4.198610,7.384838,4.965777,5.027904,5.415657,5.815669,4.993469,7.166489,5.178370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91_G6,5.070085,3.145303,11.740139,3.710909,4.543209,7.812565,9.435049,5.052995,4.173296,3.936850,...,5.580091,3.640938,7.528097,4.487142,4.815946,5.427565,6.630970,5.637030,7.638832,4.106044
91_G8,5.582152,3.388021,11.904946,4.238842,4.335617,7.562667,9.253513,5.647416,3.684547,3.426587,...,6.065526,4.095702,7.463658,4.403778,4.514032,5.128584,6.499539,5.508905,7.559311,4.054981
91_H4,4.637305,3.317164,11.829372,3.931644,4.721855,7.362074,9.319269,4.730784,4.011774,3.525641,...,6.059745,4.146232,7.588047,4.534945,4.580815,5.009048,7.553446,5.343948,7.698792,5.032119
91_H5,5.225846,3.532536,12.066123,3.912106,3.238555,7.522778,9.406782,5.134564,4.228482,3.849821,...,5.716083,3.815669,7.516924,4.190327,4.684986,5.447076,6.164449,5.454917,7.874458,4.698362
